In [133]:
import numpy as np
import random
import json
import math


class HalfSpinRBM():
    '''
    use +1/-1 to encode the spin-1/2 up/down state
    '''
    def __init__(self, visible_num, hidden_num, order):
        # 'order' is used to initialize the parameters (rescale)
        self.v_num, self.h_num = visible_num, hidden_num
        self.bias_a = self._random_complex(visible_num, order)
        self.bias_b = self._random_complex(hidden_num, order)
        self.weight_w = self._random_complex((hidden_num,visible_num), order)
        self.state = np.random.choice([1,-1], visible_num)
        
        self.template = list(range(visible_num))
    
    
    def SR(self, hamiltonian, learning_rate=0.1, regular_weight=0.01, markov_eq_step=1000, sample_num=2000, epoch=20, obs=''):
        # obs 可以存放很多可观测量,每个都有对应的名字 利用字典型(先不实现)
        for epo in range(epoch):
            for i in range(markov_eq_step):
                self._sampling_once()
            
            # SR process
            list_H_loc = []
            rank = self.v_num + self.h_num + self.v_num*self.h_num
            dkdk = np.zeros((rank,rank), dtype=np.complex_)
            dk = np.zeros(rank, dtype=np.complex_)
            dkH = np.zeros(rank, dtype=np.complex_)
            # order convention: (w, v, h)
            
            for i in range(sample_num):
                self._sampling_once()
                phi_state = self._wf(self.state)
                list_H_loc.append(self._local_observable(hamiltonian, self.state))
                phi_a = self.state / phi_state 
                phi_b = np.tanh(self._effective_angles(self.state)) / phi_state
                phi_w = np.outer(phi_b, self.state) / phi_state
                phi_parameters = np.concatenate((phi_w, self.state, phi_b), axis=None)
                dkdk += np.outer(phi_parameters, phi_parameters)
                dk += phi_parameters
                dkH += phi_parameters * list_H_loc[-1]
            
            dkdk = dkdk / sample_num
            dk = dk / sample_num
            dkH = dkH / sample_num
            
            ave_H = np.mean(list_H_loc) # np.real()
            
            matrix_S = dkdk - np.outer(dk, dk) # matrix S is symmetric, how to use this fact in pinv?
            vector_f = ave_H * dk - dkH
            matrix_invS = np.linalg.pinv(matrix_S+regular_weight*np.identity(rank))
            dev_parameters = np.inner(matrix_invS, vector_f)
            self.weight_w += learning_rate * np.reshape(dev_parameters[0:self.v_num*self.h_num], (self.h_num, self.v_num))
            self.bias_a += learning_rate * dev_parameters[self.v_num*self.h_num:self.v_num*(self.h_num+1)]
            self.bias_b += learning_rate * dev_parameters[rank-self.h_num:rank]
            print(ave_H/self.v_num)

    
    def _sampling_once(self):
        phi_old = self._wf(self.state)
        site = random.choice(self.template)
        self.state[site] *= -1 # spin flip
        phi_new = self._wf(self.state)
        prob = np.real( phi_new*np.conj(phi_new) / (phi_old*np.conj(phi_old)) )
        if random.uniform(0,1) < prob:
            return 0 # accepted
        else:
            self.state[site] *= -1
            return 1 # rejected 
        
        
    def _local_observable(self, observable, state):
        # obey Python convention: index begins with 0
        # observable is a list storing the local form of an operator (with data type)
        # local observable should be a complex number? (average would be real)
        obs_loc = 0.+0.j
        phi_state = self._wf(state)
        for term in observable:
            pos_list = term[0]
            local_degree = len(pos_list)
            operator = term[1]
            config = [state[pos] for pos in pos_list]
            row = self._convertUtoV(config)
            eles = operator[row][:]
            for i in range(len(eles)):
                config_prime = self._convertVtoU(i, local_degree)
                state_prime = state.copy()
                for k in range(local_degree):
                    state_prime[pos_list[k]] = config_prime[k]
                phi_state_prime = self._wf(state_prime)
                obs_loc += phi_state_prime * eles[i] / phi_state
        return obs_loc
            
            
    def _convertUtoV(self, config_loc):
        # config_loc is a list of +1/-1 storing the local configuration
        # [1,-1,1] -> 2
        temp = (1+np.array(config_loc))/2
        temp = ''.join(str(int(1-e)) for e in temp)
        return int(temp, 2)
        
        
    def _convertVtoU(self, pos, length):
        temp = bin(pos)[2:].zfill(length)
        config = []
        for i in temp:
            config.append( 2*(1-int(i))-1 )
        return config
        
        
    def _wf(self, state):
        return np.exp(np.inner(self.bias_a, state)) * np.prod(2*np.cosh(self._effective_angles(state)))
        
        
    def _effective_angles(self, state):
        return self.bias_b + np.inner(self.weight_w, self.bias_a)
        
        
    def _random_complex(self, size, order):
        real_part = (np.random.random(size)-0.5) * math.pow(10, order)
        imag_part = (np.random.random(size)-0.5) * math.pow(10, order)
        return real_part+1.j*imag_part

In [134]:
N = 4
alpha = 2
M = alpha * N
h = 0.
sx = np.array([[0,1],[1,0]])
sz = np.array([[1,0],[0,-1]])
szz = np.kron(sz,sz)
hamiltonian = []
for i in range(N):
    hamiltonian.append([[i],-h*sx])
    hamiltonian.append([[i,(i+1)%N],-szz])
model = HalfSpinRBM(N,M,-2)
model.SR(hamiltonian, learning_rate=0.1, sample_num=2000, regular_weight=0.001, epoch=50)

-0.0095
0.003
-0.0085
0.008
0.0155
-0.0275
-0.008499999999999999
0.013000000000000001
-0.014
-0.027999999999999994
-0.0495
-0.1155
-0.183
-0.2745
-0.45
-0.6165
-0.7499999999999999
-0.8975
-0.9400000000000001
-0.981
-0.972
-0.998
-0.9975
-0.999
-1.0
-0.9935
-0.9955
-1.0
-1.0
-1.0
-0.9975
-0.9995
-0.9999999999999999
-0.9999999999999999
-0.9999999999999999
-0.9979999999999999
-0.9999999999999999
-0.9999999999999999
-0.9944999999999999
-0.997
-1.0
-1.0
-0.9995
-1.0
-1.0
-1.0
-0.9985
-0.9975
-1.0
-1.0
